In [159]:
import pandas as pd
import re
import numpy as np
from fuzzywuzzy import process

In [171]:
raw_projections = pd.read_csv('historical_projections.csv')
raw_salaries = pd.read_csv('historical_salaries.csv')
raw_odds = pd.read_csv('historical_odds.csv')

In [3]:
raw_projections.head()

,ATT_PASS,ATT_RUSH,CMP,FF,FG,FGA,FL,FPTS,FR,INT,...,TDS_REC,TDS_RUSH,Team,Week,XPT,YDS_AGN,YDS_PASS,YDS_REC,YDS_RUSH,Year
0,30.1,6.3,19.9,0.0,0.0,0.0,0.2,20.6,0.0,0.0,...,0.0,0.2,SEA,1,0.0,0.0,245.0,0.0,34.9,2016
1,36.9,3.7,23.3,0.0,0.0,0.0,0.2,20.4,0.0,0.0,...,0.0,0.1,GB,1,0.0,0.0,274.7,0.0,17.4,2016
2,38.7,4.0,23.5,0.0,0.0,0.0,0.3,20.3,0.0,0.0,...,0.0,0.1,IND,1,0.0,0.0,275.9,0.0,17.8,2016
3,40.4,1.3,27.0,0.0,0.0,0.0,0.2,19.7,0.0,0.0,...,0.0,0.1,NO,1,0.0,0.0,307.2,0.0,2.2,2016
4,37.1,2.1,23.4,0.0,0.0,0.0,0.3,18.5,0.0,0.0,...,0.0,0.0,OAK,1,0.0,0.0,274.5,0.0,7.7,2016


In [4]:
raw_salaries.head()

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,DK points,DK salary
0,1,2016,1409,"Luck, Andrew",QB,ind,h,det,38.50,8300.0
1,1,2016,1151,"Brees, Drew",QB,nor,h,oak,35.42,8100.0
2,1,2016,1254,"Smith, Alex",QB,kan,h,sdg,32.02,6500.0
3,1,2016,1340,"Stafford, Matthew",QB,det,a,ind,29.10,7200.0
4,1,2016,1301,"Ryan, Matt",QB,atl,h,tam,27.36,7100.0


In [5]:
def clean_projections(df):
    # DraftKings does not use Kickers
    df = df[df['Pos'] != 'K']
    
    return df

In [6]:
projections = clean_projections(raw_projections)

In [7]:
def parse_name(name):
    if "Defense" in name:
        new_name = re.sub(' Defense', '', name)
    elif "," in name:
        split_name = name.split(',')
        new_name = split_name[1].strip() + ' ' + split_name[0].strip()
    else:
        new_name = name
    return new_name

In [139]:
def clean_salaries(df, dst_names):
    df['Player'] = df.Name.apply(lambda x: parse_name(x))
    df['Team'] = df['Team'].str.upper()
    df['Oppt'] = df['Oppt'].str.upper()
    df['Pos'] = df['Pos'].str.replace('Def', 'DST')
    df['Player'] = df['Player'].str.replace('San Diego', 'LA Chargers')
    df.loc[df['Pos']=='DST', 'Player'] = df[df['Pos']=='DST']['Player'].apply(lambda x: process.extractOne(x, dst_names)[0])
    
    def extract_limit(name, new_names, limit):
        new_name, score = process.extractOne(name, new_names)
        if score >= limit:
            return new_name
        else:
            return name
    df['Player'] = df['Player'].apply(lambda x: extract_limit(x, all_names, 90))

    return df

In [140]:
dst_names = projections[projections['Pos']=='DST']['Player'].unique().tolist()
all_names = projections['Player'].unique().tolist()
salaries = clean_salaries(raw_salaries, dst_names)

In [141]:
projections.shape

(15740, 29)

In [142]:
salaries.shape

(13608, 11)

In [176]:
sal_proj = pd.merge(projections, salaries, on=['Year', 'Week', 'Player'], how='inner', indicator=False)

In [177]:
sal_proj['team_abbrev'] = sal_proj['Team_y']
sal_proj['team_abbrev'] = sal_proj['team_abbrev'].str.replace('SDG', 'LAC')
sal_proj['team_abbrev'] = sal_proj['team_abbrev'].str.replace('GNB', 'GB')
sal_proj['team_abbrev'] = sal_proj['team_abbrev'].str.replace('JAC', 'JAX')
sal_proj['team_abbrev'] = sal_proj['team_abbrev'].str.replace('KAN', 'KC')
sal_proj['team_abbrev'] = sal_proj['team_abbrev'].str.replace('NWE', 'NE')
sal_proj['team_abbrev'] = sal_proj['team_abbrev'].str.replace('NOR', 'NO')
sal_proj['team_abbrev'] = sal_proj['team_abbrev'].str.replace('SFO', 'SF')
sal_proj['team_abbrev'] = sal_proj['team_abbrev'].str.replace('TAM', 'TB')

In [178]:
sal_proj.shape

(11195, 38)

In [179]:
sal_proj.head()

,ATT_PASS,ATT_RUSH,CMP,FF,FG,FGA,FL,FPTS,FR,INT,...,Year,GID,Name,Pos_y,Team_y,h/a,Oppt,DK points,DK salary,team_abbrev
0,30.1,6.3,19.9,0.0,0.0,0.0,0.2,20.6,0.0,0.0,...,2016,1412,"Wilson, Russell",QB,SEA,h,MIA,13.92,7900.0,SEA
1,36.9,3.7,23.3,0.0,0.0,0.0,0.2,20.4,0.0,0.0,...,2016,1252,"Rodgers, Aaron",QB,GNB,a,JAC,23.56,8500.0,GB
2,38.7,4.0,23.5,0.0,0.0,0.0,0.3,20.3,0.0,0.0,...,2016,1409,"Luck, Andrew",QB,IND,h,DET,38.50,8300.0,IND
3,40.4,1.3,27.0,0.0,0.0,0.0,0.2,19.7,0.0,0.0,...,2016,1151,"Brees, Drew",QB,NOR,h,OAK,35.42,8100.0,NO
4,37.1,2.1,23.4,0.0,0.0,0.0,0.3,18.5,0.0,0.0,...,2016,1441,"Carr, Derek",QB,OAK,a,NOR,25.36,7300.0,OAK


In [180]:
raw_odds.head()

,game_id,home/away,Week,OverUnder,Year,team,spread,ml,ID,Name,Abbreviation,Conference,Division
0,0,away,1,41.0,2016,Broncos,3.0,145,10,Denver Broncos,DEN,AFC,West
1,28,away,2,47.0,2016,Broncos,-6.5,-280,10,Denver Broncos,DEN,AFC,West
2,38,home,3,41.5,2016,Broncos,3.5,155,10,Denver Broncos,DEN,AFC,West
3,57,home,4,43.0,2016,Broncos,-3.5,-180,10,Denver Broncos,DEN,AFC,West
4,71,away,5,44.5,2016,Broncos,-4.0,-200,10,Denver Broncos,DEN,AFC,West


In [191]:
df = pd.merge(sal_proj, raw_odds, left_on=['Year', 'Week', 'team_abbrev'], right_on=['Year', 'Week', 'Abbreviation'], how='inner')

In [193]:
df.to_csv('training_data.csv', index=False)

In [143]:
test = pd.merge(projections, salaries, on=['Year', 'Week', 'Player'], how='outer', indicator=True)

In [144]:
test.head()

,ATT_PASS,ATT_RUSH,CMP,FF,FG,FGA,FL,FPTS,FR,INT,...,Year,GID,Name,Pos_y,Team_y,h/a,Oppt,DK points,DK salary,_merge
0,30.1,6.3,19.9,0.0,0.0,0.0,0.2,20.6,0.0,0.0,...,2016,1412.0,"Wilson, Russell",QB,SEA,h,MIA,13.92,7900.0,both
1,36.9,3.7,23.3,0.0,0.0,0.0,0.2,20.4,0.0,0.0,...,2016,1252.0,"Rodgers, Aaron",QB,GNB,a,JAC,23.56,8500.0,both
2,38.7,4.0,23.5,0.0,0.0,0.0,0.3,20.3,0.0,0.0,...,2016,1409.0,"Luck, Andrew",QB,IND,h,DET,38.50,8300.0,both
3,40.4,1.3,27.0,0.0,0.0,0.0,0.2,19.7,0.0,0.0,...,2016,1151.0,"Brees, Drew",QB,NOR,h,OAK,35.42,8100.0,both
4,37.1,2.1,23.4,0.0,0.0,0.0,0.3,18.5,0.0,0.0,...,2016,1441.0,"Carr, Derek",QB,OAK,a,NOR,25.36,7300.0,both


In [145]:
test.tail()

,ATT_PASS,ATT_RUSH,CMP,FF,FG,FGA,FL,FPTS,FR,INT,...,Year,GID,Name,Pos_y,Team_y,h/a,Oppt,DK points,DK salary,_merge
18296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,4481.0,"Cumberland, Jeff",TE,LAC,h,OAK,0.0,2500.0,right_only
18297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,4711.0,"Carter, Cethan",TE,CIN,a,BAL,0.0,2500.0,right_only
18298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,4637.0,"Supernaw, Phillip",TE,TEN,h,JAC,0.0,2500.0,right_only
18299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,4674.0,"LaCosse, Matt",TE,DEN,h,KAN,0.0,2500.0,right_only
18300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,4685.0,"Telfer, Randall",TE,CLE,a,PIT,0.0,2500.0,right_only


In [146]:
test.shape

(18301, 38)

In [147]:
test.columns

Index([u'ATT_PASS', u'ATT_RUSH', u'CMP', u'FF', u'FG', u'FGA', u'FL', u'FPTS',
       u'FR', u'INT', u'INTS', u'PA', u'Player', u'Pos_x', u'REC', u'SACK',
       u'SAFETY', u'TD', u'TDS_PASS', u'TDS_REC', u'TDS_RUSH', u'Team_x',
       u'Week', u'XPT', u'YDS_AGN', u'YDS_PASS', u'YDS_REC', u'YDS_RUSH',
       u'Year', u'GID', u'Name', u'Pos_y', u'Team_y', u'h/a', u'Oppt',
       u'DK points', u'DK salary', u'_merge'],
      dtype='object')

In [148]:
test._merge.value_counts()

both          11195
left_only      4565
right_only     2541
Name: _merge, dtype: int64

In [138]:
test._merge.value_counts()

both          11057
left_only      4703
right_only     2675
Name: _merge, dtype: int64

In [151]:
test[test['_merge']!='both'][['Player', 'DK salary']].sort_values('DK salary', ascending=False).head(20)

,Player,DK salary
15812,Dez Bryant,8500.0
15981,Dez Bryant,8400.0
15887,Dez Bryant,8400.0
17991,Mark Ingram,8300.0
16721,DeMarco Murray,8200.0
18156,Mark Ingram,8200.0
18209,Mark Ingram,8100.0
17927,Mark Ingram,8100.0
18042,Mark Ingram,7900.0
17705,Dez Bryant,7800.0


In [154]:
test[test['Player'].str.contains('Dez')][['Player', '_merge']]

,Player,_merge
15812,Dez Bryant,right_only
15887,Dez Bryant,right_only
15981,Dez Bryant,right_only
16440,Dez Bryant,right_only
16563,Dez Bryant,right_only
16651,Dez Bryant,right_only
16752,Dez Bryant,right_only
16849,Dez Bryant,right_only
16939,Dez Bryant,right_only
17046,Dez Bryant,right_only


In [115]:
player_x_missing = set(test[test._merge == 'left_only']['Player'].tolist())
player_y_missing = set(test[test._merge == 'right_only']['Player'].tolist())

In [116]:
SIDE_DICT = {'left_only': player_y_missing, 'right_only': player_x_missing}
# SIDE_DICT = {'left_only': player_y_missing}
# test = test.head(100)

for k, v in SIDE_DICT.iteritems():
    test[k] = test[test._merge == k]['Player'].apply(lambda x: process.extractOne(x, v))
    test[k + '_player'] = test[test._merge == k][k].apply(lambda x: x[0])
    test[k + '_score'] = test[test._merge == k][k].apply(lambda x: x[1])

In [118]:
test[test['_merge']!='both'][['left_only_score', 'right_only_score']].head(10)

,left_only_score,right_only_score
22,95.0,NaN
29,100.0,NaN
30,53.0,NaN
31,100.0,NaN
32,86.0,NaN
33,100.0,NaN
34,100.0,NaN
35,63.0,NaN
36,72.0,NaN
37,64.0,NaN


In [128]:
test[test['right_only_score'].between(90, 99)][['Player', 'right_only']].head(20)

,Player,right_only
15762,Robert Griffin III,"(Robert Griffin, 95)"
15810,Ted Ginn Jr.,"(Ted Ginn, 95)"
15856,E.J. Manuel,"(EJ Manuel, 90)"
15904,Ted Ginn Jr.,"(Ted Ginn, 95)"
15993,Ted Ginn Jr.,"(Ted Ginn, 95)"
16101,Ted Ginn Jr.,"(Ted Ginn, 95)"
16190,Ted Ginn Jr.,"(Ted Ginn, 95)"
16235,E.J. Manuel,"(EJ Manuel, 90)"
16236,A.J. McCarron,"(AJ McCarron, 92)"
16272,Ted Ginn Jr.,"(Ted Ginn, 95)"


In [127]:
test[test['left_only_score'].between(90, 99)][['Player', 'left_only']].head(20)

,Player,left_only
22,Robert Griffin,"(Robert Griffin III, 95)"
38,EJ Manuel,"(E.J. Manuel, 90)"
41,AJ McCarron,"(A.J. McCarron, 92)"
182,Dan Vitale,"(Danny Vitale, 91)"
254,Ted Ginn,"(Ted Ginn Jr., 95)"
311,Damiere Byrd,"(Damier Byrd, 96)"
536,EJ Manuel,"(E.J. Manuel, 90)"
537,AJ McCarron,"(A.J. McCarron, 92)"
561,Robert Griffin,"(Robert Griffin III, 95)"
678,Dan Vitale,"(Danny Vitale, 91)"


In [70]:
test[test['left_only'].notnull()][['Player', 'left_only', 'left_only_score']].head(20)

,Player,left_only,left_only_score
22,Robert Griffin,"(Robert Griffin III, 95)",95.0
29,Sam Bradford,"(Sam Bradford, 100)",100.0
30,Paxton Lynch,"(Jordan Payton, 53)",53.0
31,Josh McCown,"(Josh McCown, 100)",100.0
32,Jared Goff,"(Jared Abbrederis, 86)",86.0
33,Geno Smith,"(Geno Smith, 100)",100.0
34,Mike Glennon,"(Mike Glennon, 100)",100.0
35,Chad Henne,"(Rashad Jennings, 63)",63.0
36,Landry Jones,"(Andre Johnson, 72)",72.0
37,Taylor Heinicke,"(Tyrod Taylor, 64)",64.0


In [ ]:
test[test['left_only'].notnull()].Pos_x.value_counts()

In [72]:
test[test['right_only'].notnull()].Pos_y.value_counts()

DST    1024
WR      974
RB      830
TE      690
QB      181
Name: Pos_y, dtype: int64

In [73]:
test[(test['left_only'].notnull()) & (test['Pos_x']=='DST')][['Player', 'left_only', 'left_only_score']].head(10)

,Player,left_only,left_only_score
468,Carolina Panthers,"(Carolina, 90)",90.0
469,Green Bay Packers,"(Green Bay, 90)",90.0
470,Los Angeles Rams,"(Los Angeles, 95)",95.0
471,Kansas City Chiefs,"(Kansas City, 90)",90.0
472,Seattle Seahawks,"(Seattle, 90)",90.0
473,Minnesota Vikings,"(Minnesota, 90)",90.0
474,Philadelphia Eagles,"(Philadelphia, 90)",90.0
475,Houston Texans,"(Houston, 90)",90.0
476,Cincinnati Bengals,"(Cincinnati, 90)",90.0
477,Detroit Lions,"(Detroit, 90)",90.0


In [74]:
test[(test['left_only'].notnull()) & (test['Pos_x']=='DST')]['left_only_score'].value_counts()

90.0    928
95.0     64
87.0     32
Name: left_only_score, dtype: int64

In [75]:
test[(test['left_only'].notnull()) & (test['Pos_x']=='DST') & (test['left_only_score'] == 87)][['Player', 'left_only']]

,Player,left_only
488,New York Jets,"(New York J, 87)"
978,New York Jets,"(New York J, 87)"
1464,New York Jets,"(New York J, 87)"
1929,New York Jets,"(New York J, 87)"
2357,New York Jets,"(New York J, 87)"
2819,New York Jets,"(New York J, 87)"
3264,New York Jets,"(New York J, 87)"
3655,New York Jets,"(New York J, 87)"
4048,New York Jets,"(New York J, 87)"
4466,New York Jets,"(New York J, 87)"


In [76]:
test[(test['right_only'].notnull()) & (test['Pos_y']=='DST')]['Player'].unique()

array(['Minnesota', 'San Francisco', 'Miami', 'Philadelphia', 'Seattle',
       'Buffalo', 'New York J', 'Houston', 'Carolina', 'Denver',
       'Arizona', 'Baltimore', 'Cincinnati', 'Dallas', 'Green Bay',
       'Pittsburgh', 'Chicago', 'Tennessee', 'LA Chargers', 'Tampa Bay',
       'New England', 'Oakland', 'Cleveland', 'Jacksonville',
       'Kansas City', 'Atlanta', 'Los Angeles', 'New York G', 'Detroit',
       'Washington', 'Indianapolis', 'New Orleans', 'LA Rams'],
      dtype=object)

In [77]:
test[(test['left_only'].notnull()) & (test['Pos_x']=='DST')]['Player'].unique()

array(['Carolina Panthers', 'Green Bay Packers', 'Los Angeles Rams',
       'Kansas City Chiefs', 'Seattle Seahawks', 'Minnesota Vikings',
       'Philadelphia Eagles', 'Houston Texans', 'Cincinnati Bengals',
       'Detroit Lions', 'Pittsburgh Steelers', 'Denver Broncos',
       'Cleveland Browns', 'Arizona Cardinals', 'Indianapolis Colts',
       'Tampa Bay Buccaneers', 'New England Patriots', 'Buffalo Bills',
       'Washington Redskins', 'Atlanta Falcons', 'New York Jets',
       'New York Giants', 'Oakland Raiders', 'Jacksonville Jaguars',
       'Tennessee Titans', 'San Francisco 49ers', 'Chicago Bears',
       'New Orleans Saints', 'Los Angeles Chargers', 'Baltimore Ravens',
       'Dallas Cowboys', 'Miami Dolphins'], dtype=object)

In [78]:
test[(test['right_only'].notnull()) & (test['Pos_y']=='DST') & (test['right_only_score'] == 86)][['Player', 'right_only']]

,Player,right_only
15872,LA Chargers,"(Los Angeles Chargers, 86)"
15995,LA Chargers,"(Los Angeles Chargers, 86)"
16120,LA Chargers,"(Los Angeles Chargers, 86)"
16258,LA Chargers,"(Los Angeles Chargers, 86)"
16376,LA Chargers,"(Los Angeles Chargers, 86)"
16487,LA Chargers,"(Los Angeles Chargers, 86)"
16631,LA Chargers,"(Los Angeles Chargers, 86)"
16742,LA Chargers,"(Los Angeles Chargers, 86)"
16875,LA Chargers,"(Los Angeles Chargers, 86)"
17031,LA Chargers,"(Los Angeles Chargers, 86)"


In [79]:
test[(test['right_only'].notnull()) & (test['Pos_y']=='DST')]['right_only_score'].value_counts()

90.0    896
86.0     48
95.0     48
87.0     32
Name: right_only_score, dtype: int64

In [47]:
test[(test['right_only'].notnull()) & (test['Pos_y']=='DST')][['Player', 'right_only']].head(10)

,Player,right_only
15854,Minnesota,"(Minnesota Vikings, 90)"
15855,San Francisco,"(San Francisco 49ers, 95)"
15856,Miami,"(Miami Dolphins, 90)"
15857,Philadelphia,"(Philadelphia Eagles, 90)"
15858,Seattle,"(Seattle Seahawks, 90)"
15859,Buffalo,"(Buffalo Bills, 90)"
15860,New York J,"(New York Jets, 87)"
15861,Houston,"(Houston Texans, 90)"
15862,Carolina,"(Carolina Panthers, 90)"
15863,Denver,"(Denver Broncos, 90)"


In [116]:
test[test._merge == 'right_only'].sort_values('DK salary', ascending=False).head(20)

,ATT_PASS,ATT_RUSH,CMP,FF,FG,FGA,FL,FPTS,FR,INT,...,Year,GID,Name,Pos_y,Team_y,h/a,Oppt,DK points,DK salary,_merge
15814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2016,3891.0,"Bryant, Dez",WR,DAL,h,NYG,1.8,8500.0,right_only
16053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2016,3891.0,"Bryant, Dez",WR,DAL,h,CHI,13.0,8400.0,right_only
15924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2016,3891.0,"Bryant, Dez",WR,DAL,a,WAS,20.2,8400.0,right_only
18930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,2793.0,"Ingram, Mark",RB,NOR,a,LAR,5.6,8300.0,right_only
19207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,2793.0,"Ingram, Mark",RB,NOR,h,NYJ,32.1,8200.0,right_only
17045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2016,2831.0,"Murray, DeMarco",RB,TEN,a,IND,16.9,8200.0,right_only
19296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,2793.0,"Ingram, Mark",RB,NOR,h,ATL,15.6,8100.0,right_only
18834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,2793.0,"Ingram, Mark",RB,NOR,h,WAS,27.5,8100.0,right_only
19017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,2793.0,"Ingram, Mark",RB,NOR,h,CAR,24.2,7900.0,right_only
18488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017,3891.0,"Bryant, Dez",WR,DAL,a,SFO,19.3,7800.0,right_only


In [17]:
vc = test[test._merge == 'left_only'].Player.value_counts()
vc[vc > 20]

Devante Parker          32
Washington Redskins     32
Los Angeles Chargers    32
Baltimore Ravens        32
EJ Manuel               32
New York Jets           32
Denver Broncos          32
Minnesota Vikings       32
Houston Texans          32
Arizona Cardinals       32
Indianapolis Colts      32
AJ McCarron             32
Philadelphia Eagles     32
Cleveland Browns        32
Oakland Raiders         32
Dallas Cowboys          32
Seattle Seahawks        32
Chicago Bears           32
MarQueis Gray           32
Pittsburgh Steelers     32
Los Angeles Rams        32
Miami Dolphins          32
Rashawn Scott           32
Kansas City Chiefs      32
Carolina Panthers       32
Atlanta Falcons         32
Devin Lucien            32
New York Giants         32
Buffalo Bills           32
Tennessee Titans        32
                        ..
Darius Jennings         29
Austin Davis            29
Brandon Weeden          28
CJ Ham                  28
Nick Williams           28
Benjamin Watson         28
M

In [18]:
vc = test[test._merge == 'right_only'].Player.value_counts()
vc[vc > 20]

Jason Witten         32
Washington           32
Denver               32
Chicago              32
New Orleans          32
Arizona              32
Philadelphia         32
Detroit              32
Anthony Fasano       32
Brandon Coleman      32
Green Bay            32
New England          32
San Francisco        32
Brandon LaFell       32
Indianapolis         32
Cleveland            32
Buffalo              32
Pittsburgh           32
Carolina             32
Tennessee            32
Eric Weems           32
Kansas City          32
New York G           32
Houston              32
Cincinnati           32
Oakland              32
Miami                32
New York J           32
Marqueis Gray        32
Minnesota            32
                     ..
Atlanta              32
DeMarco Murray       31
Ted Ginn Jr.         31
Phillip Supernaw     31
Brandon Williams     30
Antonio Gates        30
Dez Bryant           29
Mike Tolbert         28
DeVante Parker       28
Dontrelle Inman      28
Jordan Todman   

In [65]:
test[test._merge == 'left_only'].sort_values('FPTS', ascending=False).head(20)

,ATT_PASS,ATT_RUSH,CMP,FG,FGA,FL,FPTS,INTS,Player,Pos_x,...,Year,GID,Name,Pos_y,Team_y,h/a,Oppt,DK points,DK salary,_merge
5836,33.0,4.2,19.9,0.0,0.0,0.2,14.5,1.0,Robert Griffin,QB,...,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6807,31.5,5.5,18.6,0.0,0.0,0.3,14.0,1.1,Robert Griffin,QB,...,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
22,30.8,3.8,19.2,0.0,0.0,0.3,13.8,0.9,Robert Griffin,QB,...,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
14710,29.4,3.0,18.0,0.0,0.0,0.2,13.4,0.7,Mitch Trubisky,QB,...,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6324,31.9,4.6,18.6,0.0,0.0,0.2,13.3,1.2,Robert Griffin,QB,...,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7292,29.2,4.3,17.4,0.0,0.0,0.2,13.3,0.8,EJ Manuel,QB,...,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
13217,27.3,4.2,15.8,0.0,0.0,0.3,12.7,0.8,Mitch Trubisky,QB,...,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9730,31.8,3.0,18.9,0.0,0.0,0.2,12.6,1.2,EJ Manuel,QB,...,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
11064,26.0,3.6,15.6,0.0,0.0,0.3,12.2,0.8,Mitch Trubisky,QB,...,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
14217,28.0,3.5,16.8,0.0,0.0,0.3,12.1,0.8,Mitch Trubisky,QB,...,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
